<a href="https://colab.research.google.com/github/bethuunltd/LSTM-Weather-Forecasting/blob/main/CNNParameterForecasting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import torch
import numpy as np
import torch.nn as nn
from sklearn.preprocessing import MinMaxScaler
from torch.utils.data import TensorDataset,DataLoader


In [3]:
df = pd.read_csv('data.csv')
df['date'] = pd.to_datetime(df['date'])
df.set_index('date')
print(df['date'])
parameter = df[['ws']]
lookback = 10
horizon = 5

scaler = MinMaxScaler()

train_len = 0.7*len(parameter)
val_len = 0.2*len(parameter)

train = parameter[:int(train_len)]
val = parameter[int(train_len):int(train_len+val_len)]
test = parameter[int(train_len+val_len):]


#The scaler has no cross-contamination between features.
train_scaled = scaler.fit_transform(train)
val_scaled = scaler.transform(val)
test_scaled = scaler.transform(test)




0       2016-01-01 00:00:00
1       2016-01-01 01:00:00
2       2016-01-01 02:00:00
3       2016-01-01 03:00:00
4       2016-01-01 04:00:00
                ...        
44683   2021-02-04 19:00:00
44684   2021-02-04 20:00:00
44685   2021-02-04 21:00:00
44686   2021-02-04 22:00:00
44687   2021-02-04 23:00:00
Name: date, Length: 44688, dtype: datetime64[ns]


In [4]:
def create_sequences(data,lookback,horizon):
  X,y = [],[]
  for i in range(len(data)-lookback-horizon):
    X.append(data[i:i+lookback])
    y.append(data[i+lookback:i+lookback+horizon])#Starts from the last datapoint of X, and goes "horizon" days more.
  return np.array(X),np.array(y)


X_train,y_train = create_sequences(train_scaled,lookback,horizon)
X_val,y_val = create_sequences(val_scaled,lookback,horizon)
X_test,y_test = create_sequences(test_scaled,lookback,horizon)
X_train = torch.from_numpy(X_train).float()
y_train = torch.from_numpy(y_train).float()
X_val = torch.from_numpy(X_val).float()
y_val = torch.from_numpy(y_val).float()
X_test = torch.from_numpy(X_test).float()
y_test = torch.from_numpy(y_test).float()



In [5]:
class CNN_LSTM(nn.Module):
    def __init__(self, input_size=1, cnn_filters=56, kernel_size=3,
                 hidden_size=64, num_layers=2, horizon=5, dropout=0.3):
        super(CNN_LSTM, self).__init__()
        self.horizon = horizon
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers

        self.cnn = nn.Conv1d(in_channels=input_size,
                             out_channels=cnn_filters,
                             kernel_size=kernel_size,
                             padding=kernel_size // 2)#Kernel size padding looks back and forth from
        self.relu = nn.ReLU()
        self.cnn_dropout = nn.Dropout(p=dropout)

        self.lstm = nn.LSTM(input_size=cnn_filters,
                            hidden_size=hidden_size,
                            num_layers=num_layers,
                            batch_first=True,
                            dropout=dropout)

        self.fc_dropout = nn.Dropout(p=dropout)
        self.fc = nn.Linear(hidden_size, horizon * input_size)

    def forward(self, x):
        x = x.permute(0, 2, 1)  # (batch, features, time)
        x = self.relu(self.cnn(x))
        x = self.cnn_dropout(x)
        x = x.permute(0, 2, 1)  # (batch, time, features)

        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size, device=x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size, device=x.device)

        out, _ = self.lstm(x, (h0, c0))
        out = self.fc_dropout(out[:, -1, :])
        out = self.fc(out)
        out = out.view(-1, self.horizon, self.input_size)
        return out